In [9]:
import os
import json
import torch
import urllib.request
import urllib.parse
import numpy as np


In [10]:
def open_json(filename):
    f = open(filename, encoding='utf-8')
    res = f.read()
    data = json.loads(res)
    f.close()
    return data

In [11]:
def get_num_of_user(data):
    l = []
    for _ in data:
        if int(_) not in l:
            l.append(int(_))
    return len(l)

In [12]:
def get_num_of_case(data):
    l = []
    for _ in data:
        for __ in data[_]['cases']:
            if int(__['case_id']) not in l:
                l.append(int(__['case_id']))
    return len(l)

In [13]:
def origin_data_to_list(data):
    user = []
    case = []
    for _ in data:
        user.append(_)
        for __ in data[_]['cases']:
            case_id = int(__['case_id'])
            if case_id not in case:
                case.append(case_id)
    user.sort()
    case.sort()
    return user, case

In [14]:
def get_matrix(filename):
    data = open_json(filename)
    r = get_num_of_user(data)
    c = get_num_of_case(data)
    user, case = origin_data_to_list(data)
    mat = torch.empty(r, c)
    mark_mat = []
    for _ in data:
        i = user.index(_)
        for __ in data[_]['cases']:
            case_id = int(__['case_id'])
            final_score = __['final_score']
            j = case.index(case_id)
            mat[i][j] = final_score
            if [i, j] not in mark_mat:
                mark_mat.append([i, j])
    return mat, mark_mat

In [15]:
def biasSVD(data, k, steps, learning_rate, l, train, test):
    print(data)
    n = data.shape[0]
    m = data.shape[1]
    item = torch.randn(m, k).normal_(0, 1)
    user = torch.randn(n, k).normal_(0, 1)
    bi = torch.randn(m, 1).normal_(0, 1)
    bu = torch.randn(n, 1).normal_(0, 1)
    miu = 50
    # train = torch.load('//TODO', map_location=torch.device('cpu'))
    # _ = train.shape[0]
    _ = data.shape[0]
    print(user.shape)
    print(item.shape)
    for t in range(steps):
        print('----------round', t, '----------begin')
        user_gradient = torch.zeros(n, k)
        item_gradient = torch.zeros(m, k)
        bi_gradient = torch.zeros(m, 1)
        bu_gradient = torch.zeros(n, 1)
        for _ in train:
            i = _[0]
            j = _[1]
            error = torch.dot(user[i], item[j].t()) + bi[j] + bu[i] + miu - data[i][j]
            user_gradient[i] = torch.add(user_gradient[i], torch.mul(error, item[j]))
            item_gradient[j] = torch.add(item_gradient[j], torch.mul(error, user[i]))
            bu_gradient[i] = torch.add(bu_gradient[i], error)
            bi_gradient[j] = torch.add(bi_gradient[j], error)
        user_gradient = torch.add(user_gradient, torch.mul(l, user))
        item_gradient = torch.add(item_gradient, torch.mul(l, item))
        bu_gradient = torch.add(bu_gradient, torch.mul(l, bu))
        bi_gradient = torch.add(bi_gradient, torch.mul(l, bi))
        print('----------gradient has trained----------')
        user = torch.sub(user, learning_rate * user_gradient)
        item = torch.sub(item, learning_rate * item_gradient)
        bu = torch.sub(bu, learning_rate * bu_gradient)
        bi = torch.sub(bi, learning_rate * bi_gradient)
        error = 0
        count = len(test)
        for _ in test:
            i = _[0]
            j = _[1]
            error += torch.pow(data[i][j]-torch.dot(user[i], item[j].t())-miu-bi[j]-bu[i], 2)
        print('----------round', t, '----------rmse is: ', torch.sqrt((1/count)*error))
        print('----------round', t, '----------train end')

In [ ]:
def validCross()

In [18]:
mat, mark_mat = get_matrix('test_data.json')
t = np.random.rand(len(mark_mat), 1)
# print(t[0])
train = [mark_mat[_] for _ in range(len(mark_mat)) if t[_][0] >= 0.2]
test = [mark_mat[_] for _ in range(len(mark_mat)) if t[_][0] < 0.2]
print(len(train))
print(len(test))
biasSVD(mat, 15, 100, 0.0001, 0.1, train, test)

34239
8543
tensor([[ 0.0000e+00,  0.0000e+00,  1.4396e-38,  ...,  1.0000e+02,
          1.0000e+02,  1.0000e+02],
        [ 1.3563e-19,  2.1701e-18,  7.3174e+28,  ...,  1.8993e+28,
          3.2180e+21,  5.5595e+31],
        [ 6.8311e+22, -2.4957e-17, -1.4662e-35,  ...,  2.5038e-12,
          1.3563e-19,  1.3563e-19],
        ...,
        [ 4.8601e+30,  1.5766e-19,  7.7456e+31,  ...,  1.0000e+02,
          1.0000e+02,  1.0000e+02],
        [ 1.6936e-16,  1.5755e-04,  2.9463e+03,  ...,  1.0000e+02,
          1.0000e+02,  1.0000e+02],
        [ 1.8179e+31,  6.7346e+22,  1.8950e+31,  ...,  1.0000e+02,
          1.0000e+02,  1.0000e+02]])
torch.Size([271, 15])
torch.Size([882, 15])
----------round 0 ----------begin
----------gradient has trained----------
----------round 0 ----------rmse is:  tensor([48.9941])
----------round 0 ----------train end
----------round 1 ----------begin
----------gradient has trained----------
----------round 1 ----------rmse is:  tensor([48.0728])
----------rou